<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/Processing_CrowdFlower_using_our_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import re
import string


from scipy import stats
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from sklearn.utils import shuffle 
from sklearn.model_selection import KFold

In [6]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

In [7]:
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

In [8]:
from google.colab import  drive

drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_rows',1000)

#Import Data

In [10]:
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower.csv')

df.head()

,emotion,tweets
0,empty,@tiffanylue i know i was listenin to bad habit earlier and i started freakin at his part =[
1,sadness,Layin n bed with a headache ughhhh...waitin on your call...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,"@dannycastillo We want to trade with someone who has Houston tickets, but no one will."


In [11]:
df['emotion'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: emotion, dtype: int64

#Methods to convert Data

In [12]:
def is_camel_case(s):
    return s != s.lower() and s != s.upper() and ("_" and " ") not in s

In [13]:
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

In [14]:
def convert_emojis(tweet):
  for emot in UNICODE_EMO:
    # print("_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    tweet = tweet.replace(emot, " sbfs".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
  
  return tweet


In [15]:
 def convert_emoticons(tweet):
  tweet = tweet
  for emot in EMOTICONS:
      tweet = re.sub(u'('+emot+')', " ".join(EMOTICONS[emot].replace(",","").split()), tweet)
  return tweet

In [16]:
print(convert_emoticons(convert_emojis((":'‑) :') 🙂 😊 😀 😁 "))))

Tears of happiness Tears of happiness slightly_smiling_face smiling_face_with_smiling_eyes grinning_face grinning_face_with_smiling_eyes 


In [17]:
def clean_hashtags(tweet):
  words_in_tweet = tweet.split(" ")
  temp = []
  for i in words_in_tweet:
    if "#" in i:
      i = i.replace("#","")
      if "_" in i:
        i = i.replace("_"," ")
        temp.append(i)
      elif is_camel_case(i):
        # print("camel case block")
        temp.append(" ".join(camel_case_split(i)))
      else:
        temp.append(i)
    else:
      temp.append(i)
  
  return " ".join(temp)



In [18]:
def clean_mentions(tweet):
    tweet = re.sub(r"@[A-Za-z0-9_]+","", tweet)
    return tweet

In [19]:
def clean(tweet): 
            
    # Special characters
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"China\x89Ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"å£3million", "3 million", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    
    # Contractions
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)   
            
    # Character entity references
    tweet = re.sub(r"&gt;", "greater than", tweet)
    tweet = re.sub(r"&lt;", "less than", tweet)
    tweet = re.sub(r"&amp;", "and", tweet)
    
    # Typos, slang and informal abbreviations
    tweet = re.sub(r"w/e", "whatever", tweet)
    tweet = re.sub(r"w/", "with", tweet)
    tweet = re.sub(r"USAgov", "USA government", tweet)
    tweet = re.sub(r"recentlu", "recently", tweet)
    tweet = re.sub(r"Ph0tos", "Photos", tweet)
    tweet = re.sub(r"amirite", "am I right", tweet)
    tweet = re.sub(r"exp0sed", "exposed", tweet)
    tweet = re.sub(r"<3", "love", tweet)
    tweet = re.sub(r"amageddon", "armageddon", tweet)
    tweet = re.sub(r"Trfc", "Traffic", tweet)
               
    # Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
        
    # Words with punctuations and special characters
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    for p in punctuations:
        tweet = tweet.replace(p, f' {p} ')
        
    # ... and ..
    tweet = tweet.replace('...', ' ... ')
    if '...' not in tweet:
        tweet = tweet.replace('..', ' ... ')      
        
    # Acronyms
    tweet = re.sub(r"MH370", "Malaysia Airlines Flight 370", tweet)
    tweet = re.sub(r"mÌ¼sica", "music", tweet)
    tweet = re.sub(r"okwx", "Oklahoma City Weather", tweet)
    tweet = re.sub(r"arwx", "Arkansas Weather", tweet)    
    tweet = re.sub(r"gawx", "Georgia Weather", tweet)  
    tweet = re.sub(r"scwx", "South Carolina Weather", tweet)  
    tweet = re.sub(r"cawx", "California Weather", tweet)
    tweet = re.sub(r"tnwx", "Tennessee Weather", tweet)
    tweet = re.sub(r"azwx", "Arizona Weather", tweet)  
    tweet = re.sub(r"alwx", "Alabama Weather", tweet)
    tweet = re.sub(r"wordpressdotcom", "wordpress", tweet)    
    tweet = re.sub(r"usNWSgov", "United States National Weather Service", tweet)
    tweet = re.sub(r"Suruc", "Sanliurfa", tweet)   


    #fancy words
    tweet = re.sub(r"fu[* ]*k", "fuck", tweet)
    tweet = re.sub(r"[o|O]+[h|H]+", "oh", tweet)
    tweet = re.sub(r"[M|y]+[y|Y]+", "my", tweet)
    tweet = re.sub(r"[G|g]+[o|O]+[d|D]+", "god", tweet)
    tweet = re.sub(r"[h|H]+[i|I]+", "hi", tweet)
    tweet = re.sub(r"[o|O]+[k|K]+", "ok", tweet)
    tweet = re.sub(r"[g|G]+[o|O]+[d|D]+", "good", tweet)
    tweet = re.sub(r"[m|M]+[o|O]+[r|R]+[n|N]+[i|I]+[n|N]+[g|G]+", "morning", tweet)
    tweet = re.sub(r"[h|H]+[e|E]+[l|L]+[o|O]+", "hello", tweet)
    tweet = re.sub(r"[b|B]+[y|Y]+[e|E]+", "bye", tweet)
    #design
    tweet = re.sub(r"[0-9]+[PM|AM]", "", tweet)

    return tweet

In [20]:
print(clean("Hhhhheeeeeelllllllllllooooooooooooooooo dosto, goood morninggggggggggggggggG"))

hello dosto, good morning


In [21]:
#Function to remove Punctuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])# It will discard all punctuations
    return text_nopunct


In [22]:
def convert_label(label):
    if label == "empty":return 0 # neutral
    elif label == "sadness":return 2 # sad
    elif label == "enthusiasm":return 1 # happy
    elif label == "neutral":return 0 # neutral
    elif label == "worry":return 2 # sad
    elif label == "surprise":return 1 # happy
    elif label == "love":return 1 # happy
    elif label == "fun":return 1 # happy
    elif label == "hate":return 3 #hate
    elif label == "happiness":return 1 # happy
    elif label == "boredom":return 0 # neutral
    elif label == "relief":return 1 # happy
    elif label == "anger":return 4 #anger

#Applying Methods to Data Frame

In [23]:
def apply_methods(df):

  df['tweets'] = df['tweets'].apply(lambda tweet : " ".join(convert_emojis(tweet).split("_")))

  df['tweets'] = df['tweets'].apply(lambda tweet : convert_emoticons(tweet))

  df['tweets'] = df['tweets'].apply(lambda tweet : clean_hashtags(tweet))

  df['tweets'] = df['tweets'].apply(lambda tweet : clean_mentions(tweet))

  df['tweets'] = df['tweets'].apply(lambda tweet: clean(tweet))

  df['tweets'] = df['tweets'].apply(lambda tweet: remove_punct(tweet))

  df['emotion'] = df['emotion'].apply(lambda label: convert_label(label))

  df.head()

  return df

In [24]:
df = apply_methods(df)

In [25]:
df.head()

,emotion,tweets
0,0,i know i was listenin to bad habit earlier and i started freakin at his part
1,2,Layin n bed with a headache ughhhh waitin on your call
2,2,Funeral ceremony gloomy friday
3,1,wants to hang out with friends SOON
4,0,We want to trade with someone who has Houston tickets but no one will


# Save Clean Data

In [27]:
df.to_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower_clean_our_combined.csv',index=None)

#Stratified Sampling

In [ ]:
demo = df

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)

In [ ]:
sss.get_n_splits(demo['tweets'], demo['emotion'])

1

In [ ]:
df_train= [] 
df_test =[]

In [ ]:
for train_index, test_index in sss.split(demo['tweets'], demo['emotion']):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = demo['tweets'][train_index], demo['tweets'][test_index]
  y_train, y_test = demo['emotion'][train_index], demo['emotion'][test_index]

  df_train = pd.DataFrame({
      'tweets' : X_train,
      'emotion' : y_train
  })
  df_test = pd.DataFrame({
      'tweets' : X_test,
      'emotion' : y_test
  })



TRAIN: [26167 36619 28642 ... 31848 33646 13825] TEST: [14080 27123   217 ... 26570 36693 27606]


In [ ]:
df_train.to_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower_clean1_train.csv',index=None)
df_test.to_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower_clean1_test.csv',index=None)